# COLLECTING DATA
---

Nhiệm vụ của phần này là thực hiện lấy dữ liệu từ thông tin imdbid và title trong file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [31]:
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re

## 2. THAY ĐỔI PHIÊN LÀM VIỆC

In [32]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None # đk api xong áp vô nè
    },

    "Thi": {
        "part": 2,
        "api_key": None # đk api xong áp vô nè
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec'
    } 
}

In [33]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]

## 3. ĐỌC DỮ LIỆU ĐƯỢC THU THẬP TRƯỚC ĐÓ ĐỂ TÌM KIẾM THÊM

In [40]:
file_name = f"../Data/pre_data_part{part}.csv"

In [44]:
pre_data = pd.read_csv(file_name)
pre_data.info() # kiểm tra có null kiểu nan không nha k là chet o 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id/Title      629 non-null    object
 1   Release year  629 non-null    int64 
 2   Age rating    629 non-null    object
 3   Runtime       629 non-null    object
 4   Genre         629 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.7+ KB


## 4. HÀM LẤY DỮ LIỆU CỦA TRANG `OMDB API`

Hàm này trả về link từ id hoặc tiêu đề phim

In [36]:
def convert_url(item: str, year:int,  key = apiKey):
    if item.startswith("tt"):
        url = f'https://www.omdbapi.com/?i={item}&apikey={key}'
    else: 
        title = item.split(":")[-1].strip() # lấy tên sau :
        title = re.sub(r'[^a-zA-Z0-9\s]', '', title) # xóa kí tự đặc biệt
        title = '+'.join(title.split(' ')) # thêm dấu + giữa các từ
        url = f'https://www.omdbapi.com/?t={title}&y={year}&apikey={key}'

    return url

Hàm này thì thu thập dữ liệu

In [47]:
def collectMovieData(pre_data, key = apiKey):
    full_title = []
    full_rated = []
    released_time = []
    full_runtime = []
    full_genre = []
    metaScore = []
    rating = []
    vote = []
    boxOffice = []
    language = []
    award = []

    items = pre_data['Id/Title']
    release_year = pre_data['Release year']
    genre = pre_data['Genre']
    age_rating = pre_data['Age rating']
    runtime = pre_data['Runtime']

    for i in range(len(items)):
        print(f"[LOG]: Collecting data of {items[i]}")
        url = convert_url(items[i], year = release_year[i], key=key)
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                if data.get("Response") == "False": # bỏ qua những phim không thể tim thấy trong trang
                    break

                full_title.append(data.get('Title', 'N/A'))
                
                rated = age_rating[i]
                if rated == "Null":
                    rated = data.get('Rated', 'N/A')
                full_rated.append(rated)

                time = data.get('Released', 'N/A')
                if time == 'N/A':
                    time = release_year[i]
                released_time.append(time)

                run_time = runtime[i]
                if run_time == "Null":
                    run_time = data.get('Runtime', 'N/A')
                full_runtime.append(run_time)
                
                gen = data.get('Genre', 'N/A')
                if gen == 'N/A':
                    gen = genre[i]
                full_genre.append(gen)
                    
                metaScore.append(data.get('Metascore', 'N/A'))
                rating.append(data.get('imdbRating', 'N/A'))
                vote.append(data.get('imdbVotes', 'N/A'))
                boxOffice.append(data.get('BoxOffice', 'N/A'))
                language.append(data.get('Language', 'N/A'))
                award.append(data.get('Awards', 'N/A'))
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")

                time.sleep(2)

    movieDataframe = pd.DataFrame({
            "Title": full_title,
            "Release time": released_time,
            "Age rating": full_rated,
            "Runtime": full_runtime,
            "Genre": full_genre,
            "Lanuage": language,
            "Awards": award,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxOffice
        })

    return movieDataframe

này là thu thập dữ liệu nè

In [45]:
df1 = collectMovieData(pre_data, key=apiKey)
df1

[LOG]: Collecting data of tt28097834
[LOG]: Collecting data of tt10651230
[LOG]: Collecting data of The Burial
[LOG]: Collecting data of tt21807222
[LOG]: Collecting data of tt23334616
[LOG]: Collecting data of tt15140278
[LOG]: Collecting data of tt3715152
[LOG]: Collecting data of tt21638298
[LOG]: Collecting data of tt26938504
[LOG]: Collecting data of tt15783426
[LOG]: Collecting data of tt15744298
[LOG]: Collecting data of tt6722802
[LOG]: Collecting data of tt19760052
[LOG]: Collecting data of tt11828692
[LOG]: Collecting data of tt7991508
[LOG]: Collecting data of tt2906216
[LOG]: Collecting data of tt10160976
[LOG]: Collecting data of tt21103300
[LOG]: Collecting data of tt26736061
[LOG]: Collecting data of tt21276878
[LOG]: Collecting data of tt12404266
[LOG]: Collecting data of tt6166392
[LOG]: Collecting data of tt26452836
[LOG]: Collecting data of tt10366206
[LOG]: Collecting data of Mixed Baggage
[LOG]: Collecting data of tt21418018
[LOG]: Collecting data of The Lost Chron

,Title,Release time,Age rating,Runtime,Genre,Lanuage,Awards,Metascore,IMDb Rating,IMDb Votes,Domestic Revenue
0,The Mission,08 Dec 2023,PG-13,103 min,Documentary,English,11 nominations,N/A,6.4,"1,075","$58,377"
1,The Peasants,13 Oct 2023,R,114 min,"Animation, Drama, History","Polish, Latin",10 wins & 19 nominations,N/A,7.6,"6,678","$155,188"
2,The Burial,13 Oct 2023,R,126 min,Drama,English,2 wins & 4 nominations,N/A,7.1,"37,620",N/A
3,Saw X,29 Sep 2023,R,118 min,"Horror, Mystery, Thriller","English, Spanish",15 nominations,N/A,6.6,"84,301","$53,607,898"
4,Last Call for Istanbul,24 Nov 2023,TV-MA,91 min,"Drama, Romance",Turkish,1 win,N/A,5.3,"9,358",N/A
...,...,...,...,...,...,...,...,...,...,...,...
597,Poor Things,27 Feb 2024,R,141 min,"Comedy, Drama, Romance","English, French, Portuguese",Won 4 Oscars. 116 wins & 408 nominations total,N/A,7.8,"305,203","$34,553,225"
598,The Starling Girl,11 Jul 2023,R,117 min,Drama,English,1 win & 4 nominations,78,6.7,"2,623","$161,290"
599,"Santa, Maybe",18 Nov 2023,N/A,N/A,"Drama, Romance",English,N/A,N/A,6.2,355,N/A
600,Dear David,13 Oct 2023,R,94 min,"Horror, Thriller",English,N/A,N/A,4.0,"1,422",N/A


## 4. LƯU DỮ LIỆU THÔ VÀO FILE CSV

In [46]:
file_name = f"../Data/movie_data_part{str(part)}.csv"
df1.to_csv(file_name)